### Objectives

- Launch the trained YOLO model (models/yolov8n_handwritten_20241209.pt) on the images in the data folder. 
- Create 1 XML file for each image 
- Save the results in the results folder. 

INPUTS: 
- data/FOLDERS/
- models/yolov8n_handwritten_20241209.pt

OUTPUTS: 
- data/FOLDERS/json
- data/FOLDERS/xml





In [7]:
#activate the virtual environment

!poetry shell


In [3]:
import os

#os.chdir('D:/GitHub/trocr_handwritten')

os.chdir('C:/Users/marie/Github/trocr_handwritten')

PATH_REPO=os.getcwd()
PATH_DATA = os.path.join(PATH_REPO,'data/20241106_yolo/train')
PATH_MODEL = os.path.join(PATH_REPO,'models')
#.py functions to parse page using model
PATH_PARSE = os.path.join(PATH_REPO,'trocr_handwritten/parse')

parse_function=os.path.join(PATH_PARSE,'layout_ft_yolo.py')
model=os.path.join(PATH_MODEL,'yolov8n_handwritten_20241209.pt')
PATH_PARSE

'C:\\Users\\marie\\Github\\trocr_handwritten\\trocr_handwritten/parse'

In [4]:
import argparse
from pathlib import Path
import json
from os.path import join
from doclayout_yolo import YOLOv10

In [7]:
os.getcwd()

'C:\\Users\\marie\\Github\\trocr_handwritten'

In [11]:
!python trocr_handwritten/parse/layout_ft_yolo.py --path_folder data/20241106_yolo/images/train --path_model models/20241119_v2_yolov10_50_finetuned.pt --conf 0.2 --iou 0.5

Chargement du modèle models/20241119_v2_yolov10_50_finetuned.pt...
Prédiction sur les images dans data/20241106_yolo/images/train...

image 1/42 C:\Users\marie\Github\trocr_handwritten\data\20241106_yolo\images\train\FRANOM22_COLH78_0261_0232.jpg: 1024x736 3 Marges, 1 Nom, 1 Plein Texte, 5 Signatures, 2030.5ms
image 2/42 C:\Users\marie\Github\trocr_handwritten\data\20241106_yolo\images\train\FRANOM22_COLH78_0261_0233.jpg: 1024x736 2 En-têtes, 8 Marges, 2 Plein Textes, 3 Signatures, 1771.6ms
image 3/42 C:\Users\marie\Github\trocr_handwritten\data\20241106_yolo\images\train\FRANOM22_COLH78_0261_0234.jpg: 1024x736 1 Titre, 1 En-tête, 4 Signatures, 1 Table, 1801.8ms
image 4/42 C:\Users\marie\Github\trocr_handwritten\data\20241106_yolo\images\train\FRANOM22_COLH78_0261_0235.jpg: 1024x736 3 Marges, 1 Nom, 1 Plein Texte, 9 Signatures, 1786.9ms
image 5/42 C:\Users\marie\Github\trocr_handwritten\data\20241106_yolo\images\train\FRANOM22_COLH78_0261_0236.jpg: 1024x736 2 En-têtes, 8 Marges, 2 Plei

Break .json file into XML, one per image

1. Load JSON Data: The code reads the JSON file containing YOLO predictions.
Create Output Directory: It creates a directory for the XML files if it doesn't already exist.
3. Create XML Function: A function create_xml is defined to convert the JSON data for each image into XML format.
4. Iterate and Save: The code iterates through each image's metadata, generates the XML content, and saves it to a file named after the image (without the .jpg extension).
Note:

In [ ]:
!pip install lxml

In [15]:
# Add this code block to your Jupyter notebook

import json
import os
from lxml import etree

# Load the JSON data
with open('data/20241106_yolo/images/train/yolo_predictions.json') as json_file:
    data = json.load(json_file)

# Create the output directory for XML files
output_dir = 'data/20241106_yolo/images/train/XML'
os.makedirs(output_dir, exist_ok=True)

# Create a mapping of class numbers to names
class_mapping = {str(key): value for key, value in data['_via_attributes']['region']['Classes']['options'].items()}

# Function to create XML from JSON data
def create_xml(image_id, regions):
    root = etree.Element("annotation")
    etree.SubElement(root, "filename").text = image_id

    for region in regions:
        obj = etree.SubElement(root, "object")
        # Use the class name instead of the number
        class_name = class_mapping.get(region['region_attributes']['Classes'], 'Unknown')
        etree.SubElement(obj, "name").text = class_name
        bndbox = etree.SubElement(obj, "bndbox")
        etree.SubElement(bndbox, "xmin").text = str(region['shape_attributes']['x'])
        etree.SubElement(bndbox, "ymin").text = str(region['shape_attributes']['y'])
        etree.SubElement(bndbox, "xmax").text = str(region['shape_attributes']['x'] + region['shape_attributes']['width'])
        etree.SubElement(bndbox, "ymax").text = str(region['shape_attributes']['y'] + region['shape_attributes']['height'])

    return etree.tostring(root, pretty_print=True, encoding='unicode')

# Iterate through the image metadata and create XML files
for image_id, image_data in data['_via_img_metadata'].items():
    regions = image_data['regions']
    xml_content = create_xml(image_id.split('.')[0], regions)  # Use image name without extension
    xml_file_path = os.path.join(output_dir, f"{image_id.split('.')[0]}.xml")
    
    with open(xml_file_path, 'w') as xml_file:
        xml_file.write(xml_content)

print("XML files created successfully.")

XML files created successfully.
